# Zero Shot Detection 

- Using MobileNet V3

In [1]:
def predict(image_path,classes_dict):

    # Define image preprocessing pipeline
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load and preprocess the input image
    image = Image.open(image_path)
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

    # Perform inference
    with torch.no_grad():
        output = model(input_batch)

    # Process the output probabilities
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    return probabilities

# Imports and settings

In [2]:
import torch
import json
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os

# Load ImageNet class labels
with open("/kaggle/input/sampledsailporini/labels_map.txt","r") as f:
    classes_dict = json.load(f)

# Load pre-trained MobileNetV3 model
model = models.mobilenet_v3_small(pretrained=True)
# Alternatively, you can use 'mobilenet_v3_large' for the large version

# Set model to evaluation mode
model.eval()
print("")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 73.6MB/s]

# Inference

In [3]:
!ls /kaggle/input/dsail-porini-dataset-preparation/

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


__notebook__.ipynb  __results__.html  images
__output__.json     custom.css	      single_animal_i_images.csv


In [4]:
import pandas as pd

df_impala = pd.read_csv("/kaggle/input/impact-of-background-on-cls-with-dets/single_animal_i_dets_images.csv")#"/kaggle/input/dsail-porini-dataset-preparation/single_animal_i_images.csv")

df_impalas = df_impala[df_impala["no_of_dets_yolov8n"]>=1]

impala_images = df_impalas["Filename"].to_list()

In [5]:
df_impalas

,Filename,Device,Species,Count,Sex,Latitude,Longitude,no_of_dets_yolov8n
0,2021-08-01-07-31-10.jpg,Raspberry Pi 2,IMPALA,4,"MALE,FEMALE,FEMALE,FEMALE",-0.390386,36.962348,5
1,2021-08-01-07-31-13.jpg,Raspberry Pi 2,IMPALA,2,"MALE,FEMALE",-0.390386,36.962348,3
2,2021-08-01-07-31-35.jpg,Raspberry Pi 2,IMPALA,2,FEMALE,-0.390386,36.962348,2
3,2021-08-01-07-31-41.jpg,Raspberry Pi 2,IMPALA,2,FEMALE,-0.390386,36.962348,1
5,2021-08-13-16-21-34.jpg,Raspberry Pi 2,IMPALA,1,FEMALE,-0.390386,36.962331,1
...,...,...,...,...,...,...,...,...
5245,snapshot_201910949580242.jpg,OpenMV Cam H7,IMPALA,2,"MALE,FEMALE",-0.390381,36.962333,1
5246,snapshot_201910949581191.jpg,OpenMV Cam H7,IMPALA,2,"MALE,FEMALE",-0.390381,36.962333,1
5247,snapshot_201910949582145.jpg,OpenMV Cam H7,IMPALA,2,"MALE,FEMALE",-0.390381,36.962333,1
5248,snapshot_201910949583276.jpg,OpenMV Cam H7,IMPALA,2,CAN'T TELL,-0.390381,36.962333,1


In [6]:
# Initialize an empty list to store data
data = []

In [7]:
path = "/kaggle/input/impala-no-background/images_no_back/"#"/kaggle/input/dsail-porini-dataset-preparation/images/"
idx = 0
for image in impala_images:#os.listdir("/kaggle/input/sampledsailporini/sample-dsail-porini/dataset"):
    #     if image in ["2021-12-26-15-01-36.jpg","2021-12-26-15-01-36.jpg"]:
    #         print(f"Image {image} with Error (Skip) ")
    #         #continue
    #     else:
    try:
        #predict on image
        probabilities = predict(image_path=path+image,classes_dict=classes_dict)
        # Print the top 5 predicted labels and their probabilities
        #print(image) #Filename
        top1_prob, top1_indices = torch.topk(probabilities, 1)
        ##print(top1_prob, top1_indices)
        #print(str(top1_indices.item())) #imagenet class
        ##print(classes_dict[str(top1_indices.item())])
        #print('impala' in classes_dict[str(top1_indices.item())]) #impala true or false
        #print(top1_prob.item(),"\n") #probability
        ## Generate data for each column
        column1_data = image
        column2_data = str(top1_indices.item())
        column3_data = 'impala' in classes_dict[str(top1_indices.item())]
        column4_data = top1_prob.item()

        # Append data for this row to the list
        data.append([column1_data, column2_data,column3_data, column4_data])
        idx += 1
    except Exception as e:
        # Handle the exception
        print(f"An error occurred for image : {image}: {e}")
        continue  # Continue to the next iteration of the loop

print("Images processed : ",idx)

An error occurred for image : 2021-12-25-15-02-29.jpg: [Errno 2] No such file or directory: '/kaggle/input/impala-no-background/images_no_back/2021-12-25-15-02-29.jpg'
Images processed :  1059


In [8]:
# Define column names
columns = ['Filename','Predicted_Class','Impala','probability']

# Create DataFrame
df_results = pd.DataFrame(data, columns=columns)

In [9]:
df_results.head()

,Filename,Predicted_Class,Impala,probability
0,2021-08-01-07-31-10.jpg,607,False,0.208220
1,2021-08-01-07-31-13.jpg,607,False,0.091541
2,2021-08-01-07-31-35.jpg,430,False,0.039866
3,2021-08-01-07-31-41.jpg,111,False,0.036809
4,2021-08-13-16-21-34.jpg,417,False,0.054818


In [10]:
df_results.shape

(1059, 4)

In [11]:
df_results["Impala"].value_counts()

Impala
False    1057
True        2
Name: count, dtype: int64

In [12]:
df_results["Impala"].value_counts(normalize=True)

Impala
False    0.998111
True     0.001889
Name: proportion, dtype: float64

In [13]:
df_results["probability"].mean()

0.10696955843774973

In [14]:
df_results["probability"].median()

0.050741326063871384

In [15]:
df_results[df_results["Impala"]==True]["probability"].mean()

0.5657218843698502

In [16]:
df_results[df_results["Impala"]==True]["probability"].median()

0.5657218843698502

In [17]:
df_results[df_results["Impala"]==False]["probability"].mean()

0.10610153133097186

In [18]:
df_results[df_results["Impala"]==False]["probability"].median()

0.05070894956588745

In [19]:
# #Top 5
# top5_prob, top5_indices = torch.topk(probabilities, 1)
# print(top5_prob, top5_indices)
# for i in range(len(top5_prob)):
#     print(i)
#     print(classes_dict[str(top5_indices[i].item())])
#     print(top5_prob[i].item())
#     #print(classes_dict[str(top5_indices[i])], top5_prob[i].item())
